## LETKF
- データ同化Aの講義資料をみる
- 8メンバーで走る

### アルゴリズム
$ \tilde{P^a} = [(N-1)I/\rho + dY^T R^{-1} dY]^{-1} $

$ T = \tilde{P^a} dY^T R^{-1}(y^o - \overline{dY^f}) + [(N-1) \tilde{P^a}]^{1/2} $

$ X^a = x^f + dX^f T$

### localiztion
- 各点で分けて解析を行う．
- その際に

### Question
- 

## LETKFのdebug
TODO
- [ ] LETKF実装方針の検討
    - [ ] EnSRF(ETKF)を拡張
    - [ ] 新たにLETKFを実装
- [ ] その他はちゃんと動いているか確認
- [ ] localizationの方法の確立

In [1]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# モジュールの読み込み
import sys
sys.path.append('./')
from utils import make_lorenz96, rk4, euler, error_series_kf, search_optimal_param, plot_error_KF, estimate_error_with_params

from numpy import sqrt, trace, zeros, identity, exp, random
from numpy.random import multivariate_normal, choice
from numpy.linalg import inv
import scipy
from scipy.linalg import sqrtm

In [2]:
# Lorenz96の設定
J = 40
F = 8
lorenz = make_lorenz96(F)

# 同化step
# 時間発展は0.01ごとに行う
dt = 0.05

# モデルの遷移関数(非線形)
# 0.01ずつ時間発展させる
# dtは同化step
def M(x, dt):
    for i in range(int(dt/0.01)):
        x = rk4(0, x, 0.01, lorenz)
    return x

# 単位行列
I = np.identity(J)

# 観測
H = I

# モデル誤差共分散, 最初は完全モデルを仮定
Q = np.zeros((J, J))

# 観測誤差共分散, 後で定数倍の変化をさせる.
R = I

# 観測値と真値
end_step = 500 # 開発用
y = np.load('data/obs_atr.npy')
true = np.load('data/true_atr.npy')

# KFの初期値
np.random.seed(1)
x_0 = true[np.random.randint(len(true)-1)]
P_0 = 25*I

In [ ]:
class LETKF:
    def __init__(self, M, H, Q, R, y, x_0, P_0, m=40, dt=0.05, alpha=1, sigma=3):
        self.M = M
        self.H = H
        self.Q = Q
        self.R = R
        self.y = y
        self.m = m # アンサンブルメンバー数
        self.dt = dt
        
        # 実装で技術的に必要
        self.dim_x = Q.shape[0]
        self.I = identity(m)
        
        self.alpha = alpha # inflation用の定数
        self.sigma = sigma

        # filtering実行用
        self.x = [] # 記録用
        self.trP = []

        self._initialize(x_0, P_0, m)

  #　初期状態
    def _initialize(self, x_0, P_0, m):
        random.seed(0)
        self.X = x_0 + multivariate_normal(np.zeros(self.dim_x), P_0, m) # (m, J)
        self.x_mean = self.X.mean(axis=0)
    
  # 逐次推定を行う
    def forward_estimation(self):
        for y_obs in self.y:
            self._forecast()
            self._update(y_obs)

    # 更新/解析
    def _update(self, y_obs):
        X_f=self.X; x_f=self.x_mean; alpha = self.alpha; H = self.H; R = self.R; m = self.m; I = self.I
        
        for i in range(40):
            # dX, dYを計算
            X_f=self.X; x_f=X_f.mean(axis=0)
            dX_f = X_f - x_f # (m, dim_x)
            dX_f = sqrt(alpha)*dX_f # inflation
            dY = (H@dX_f.T).T # (m, dim_y)
        
            # Kalman gain
            K = dX_f.T@dY@inv(dY.T@dY + (m-1)*R) # (dim_x, dim_y)
            K = (self._rho(i)*K.T).T # localization
            
            # 平均を更新
            dy = self._restrict_observation(i, y_obs - H@x_f)
            x_a = x_f + K@(dy) # dim_x

            # dXを変換, I - dY^t(dYdY^t + (m-1)R)dYの平方根をとる
            S = self._symmetric(I - dY@inv(dY.T@dY + (m-1)*R)@dY.T)
            T = np.real(sqrtm(S)) # (m, m)
            self.X = x_a + (dX_f.T@T).T # (m, dim_x)
            
        self.X = X_f
        
        # 記録: 更新した値のアンサンブル平均xを保存, 推定誤差共分散P_fのtraceを保存
        self.x.append(self.X.mean(axis=0))
#         self.trP.append(sqrt(trace(dX_f.T@dX_f)/40))

    # 予報/時間発展
    def _forecast(self):
        # アンサンブルで x(k) 予測
        for i, s in enumerate(self.X):
            self.X[i] = self.M(s, self.dt)

        self.x_mean = self.X.mean(axis=0)
    
    def _restrict_observation(self, i, dy):
            r = zeros(40)
            r[i] = 1
            r[(i+1)%40]=1; r[(i-1)%40]=1
            r[(i+2)%40]=1; r[(i-2)%40]=1
            return r*dy
     
    def _rho(self, i):
        rho = np.array([exp(-min([(i-j)**2, (i+J-j)**2])/self.sigma) for j in range(self.dim_x)])
        return rho
    
    def _symmetric(self, S):
        return 0.5*(S + S.T)

In [ ]:
%%time
m = 20
letkf = LETKF(M, H, Q, R, y[:220], x_0, P_0, m=m, alpha=1.6, sigma=3)
letkf.forward_estimation()
plot_error_KF(true, y, [letkf.x], trPs=[])
estimate_error_with_params(true, [letkf.x], [m], 'm')

In [ ]:
# TODO: kalman_filters.pyに決定版を保存
# 未完成
# LocalEnsembleTransformKalmanFilter(LETKF)
"""
Parameters
M: callable(x, dt)
  状態遷移関数
H: ndarray(dim_y, dim_x)
  観測行列  
Q: ndarray(dim_x, dim_x)
  モデルの誤差共分散行列 
R: ndarray(dim_y, dim_y)
  観測の誤差共分散行列
x_0: 状態変数の初期値
P_0: 誤差共分散の初期値
m: アンサンブルメンバーの数
dt: 同化時間step幅
alpha: inflation factor
localization: localizationの設定
x: ndarray(dim_x)

"""
class LocalEnsembleTransformKalmanFilter:
    def __init__(self, M, H, Q, R, y, x_0, P_0, m=10, dt=0.05, alpha=1, rho=np.ones(10)):
        self.M = M
        self.H = H
        self.Q = Q
        self.R = R
        self.y = y
        self.m = m # アンサンブルメンバー数
        self.dt = dt
        
        # 実装で技術的に必要
        self.dim_x = Q.shape[0]
        self.I = identity(m)
        
        self.alpha = alpha # inflation用の定数
        self.rho = rho # localization用のベクトル sizeはm

        # filtering実行用
        self.x = [] # 記録用
        self.x_f = []
        self.trP = []

        self._initialize(x_0, P_0, m)

  #　初期状態
    def _initialize(self, x_0, P_0, m):
        self.X = x_0 + multivariate_normal(np.zeros(self.dim_x), P_0, m) # (m, J)
        self.x_mean = self.X.mean(axis=0)
    
  # 逐次推定を行う
    def forward_estimation(self):
        for y_obs in self.y:
            self._forecast()
            self._update(y_obs)

    # 更新/解析
    def _update(self, y_obs):
        x_f = self.x_mean; X_f = self.X; H = self.H; R = self.R; m = self.m; rho = self.rho; alpha = self.alpha

        # # dX, dYを計算
        dX_f = X_f - x_f # (m, dim_x)
        # dX_f = sqrt(self.alpha)*dX_f # 旧srfでのinflation
        dY = (H@dX_f.T).T # (m, dim_y)
        y_mean = dY.mean(axis=0) # dim_y
        
        # localization
#         self.C = dY@inv(R) # (m, dim_y)
        for i in range(self.dim_x):
            C = self._rho(i)*dY@inv(R) # Cの各i行にrho_iをかける (m, dim_y)

#             print(C_local@dY.T)
            P_at = inv(((m-1)/alpha)*self.I + C@dY.T) # (m, m)
            
            print(C@(y_obs - y_mean) )
            T = (P_at@C@(y_obs - y_mean) + sqrtm((m-1)*P_at).T).T # (m, m)

            print(dX_f.T@T)
            self.X[:, i] = x_f[i] + (dX_f.T@T).T[:, i] # (m, dim_x)

        # 記録: 更新した値のアンサンブル平均xを保存, 推定誤差共分散P_fのtraceを保存
        self.x.append(self.X.mean(axis=0))
        self.trP.append(sqrt(trace(dX_f.T@dX_f)/40))


    # 予報/時間発展
    def _forecast(self):
        # アンサンブルで x(k) 予測
        for i, s in enumerate(self.X):
            self.X[i] = self.M(s, self.dt)

        self.x_mean = self.X.mean(axis=0)
        self.x_f.append(self.x_mean)
     
    def _rho(self, i):
        rho = np.array([exp(-min([(i-j)**2, (i+J-j)**2])/self.sigma) for j in range(self.dim_x)])
        return rho